In [1]:
import glob, os
import shutil
import argparse
import wget

from astropy.time import Time
import numpy as np
from astropy.io import fits
import lightkurve as lk

path = os.path.dirname(os.getcwd())

In [2]:
CH = 1
quarters = np.sort(np.array(os.listdir('%s/data/EXBA/%i/' % (path, CH)), dtype=int))

# Get Qarters star and finish time

In [3]:
quarter_times_edges = {}
for i, q in enumerate(quarters):
    files = np.sort(glob.glob("%s/data/EXBA/%i/%i/*.fits.gz" % (path, CH, q)))

    times = lk.KeplerTargetPixelFile(files[0]).time
    quarter_times_edges[q] = [times.min(), times.max()]

# Get FFI obs times

In [4]:
ffi_list = np.sort(glob.glob("%s/data/ffi/kplr*_ffi-cal.fits" % (path)))
ffi_time = []
for ffi in ffi_list:
    hdr = fits.open(ffi)[CH].header
    ffi_time.append(Time(hdr['MJDSTART'], format="mjd"))

In [5]:
len(ffi_time), len(ffi_list)

(53, 53)

### find mapping between ffi and quarters

In [6]:
for q, edges in quarter_times_edges.items():
    print(q)
    print(edges[0])
    break

5
443.49088301277516


In [7]:
which_quarter = {}
for i, ffi in enumerate(ffi_time):
    print(ffi.jyear)
    for q, edges in quarter_times_edges.items():
        if ffi.mjd >= edges[0].mjd and ffi.mjd <= edges[1].mjd:
            which_quarter[i] = q

2009.312037306831
2009.3123796019713
2009.3127933815742
2009.313382867132
2009.3136681821218
2009.3142481363998
2009.3147516337303
2009.3159365290076
2009.4638562413143
2009.6325939144149
2009.7097470966187
2009.7975798076386
2009.8821609249555
2009.9589765895687
2010.0518397238604
2010.0520597709515
2010.1334580926489
2010.2127841980562
2010.3025724838878
2010.3816775100343
2010.4754958615742
2010.5531462999043
2010.6400835198904
2010.7250066908693
2010.8098180357015
2010.8918321745107
2010.9721120458316
2011.063807167803
2011.1436508387133
2011.315329343299
2011.3952733435729
2011.4823784003834
2011.567301753429
2011.6556933170157
2011.7406724929226
2011.8282811719096
2011.9130365761532
2012.009148580534
2012.0846172362492
2012.1615402544558
2012.2382956463791
2012.3285337000411
2012.41049169128
2012.4875268003832
2012.5749676168105
2012.6606738615742
2012.7565621487747
2012.8468561463108
2012.9319471760164
2013.0298494599315
2013.1034719108557
2013.1772063174265
2013.267555933032


In [8]:
which_quarter

{18: 5,
 19: 5,
 21: 6,
 22: 6,
 24: 7,
 25: 7,
 27: 8,
 28: 8,
 29: 9,
 30: 9,
 32: 10,
 33: 10,
 35: 11,
 36: 11,
 38: 12,
 39: 12,
 41: 13,
 42: 13,
 44: 14,
 45: 14,
 47: 15,
 48: 15,
 50: 16,
 51: 16}

In [9]:
quarter_times_edges[17][0].jyear, quarter_times_edges[17][1].jyear

(2013.2709832551216, 2013.357979679425)

In [12]:
ffi_list

array(['/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/kplr2009114174833_ffi-cal.fits',
       '/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/kplr2009114204835_ffi-cal.fits',
       '/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/kplr2009115002613_ffi-cal.fits',
       '/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/kplr2009115053616_ffi-cal.fits',
       '/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/kplr2009115080620_ffi-cal.fits',
       '/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/kplr2009115131122_ffi-cal.fits',
       '/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/kplr2009115173611_ffi-cal.fits',
       '/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/kplr2009116035924_ffi-cal.fits',
       '/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/kplr2009170043915_ffi-cal.fits',
       '/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/kplr2009231194831_ffi-cal.fits',


In [14]:
for ffi, q in which_quarter.items():
    if not os.path.isdir("%s/data/ffi/%i" % (path, q)):
        os.mkdir("%s/data/ffi/%i" % (path, q))
    
    name = ffi_list[ffi].split('/')[-1]
    out = "%s/data/ffi/%s/%s" % (path, str(q), name)
    print(out)
    shutil.move(ffi_list[ffi], out)

/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/5/kplr2010111125026_ffi-cal.fits
/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/5/kplr2010140101631_ffi-cal.fits
/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/6/kplr2010203012215_ffi-cal.fits
/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/6/kplr2010234192745_ffi-cal.fits
/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/7/kplr2010296192119_ffi-cal.fits
/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/7/kplr2010326181728_ffi-cal.fits
/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/8/kplr2011024134926_ffi-cal.fits
/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/8/kplr2011053174401_ffi-cal.fits
/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/9/kplr2011116104002_ffi-cal.fits
/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/9/kplr2011145152723_ffi-cal.fits
/Users/jorgemarpa/Work/BAERI/ADAP/EXBA_pathfinder/data/ffi/10/kplr2011208112727_